<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_jais_family_6p7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sitting up

In [2]:
!pip install accelerate
!huggingface-cli login
!pip install datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [3]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import json
import os
from google.colab import files
from IPython.display import display, FileLink
from datasets import load_dataset
import gc

# loading the model

In [4]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()


126

In [5]:

model_path = "inceptionai/jais-family-6p7b"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # Automatically maps the model to the GPU
    torch_dtype=torch.float16,  # Use half-precision for faster computation and reduced memory
    trust_remote_code=True
)

torch.cuda.empty_cache()  # Clear CUDA cache between operations

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/8.82G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# helper functions

In [6]:
def get_response(text, temperature, do_sample, top_p, num_return_sequences):
    encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=100)
    input_ids = encoding.input_ids.to(model.device)  # Ensure input is sent to the model's device
    attention_mask = encoding.attention_mask.to(model.device)

    generate_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        num_beams=10,
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=1.2,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode all generated sequences
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in generate_ids]


In [7]:
def generate_answer(question, num_samples):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_high_temp = get_response(
        prompt,
        temperature=1.0,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples - 1
    )

    # Generate the last answer with low temperature
    outputs_low_temp = get_response(
        prompt,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1
    )

    # Combine the results
    responses = [
        output.split("Answer:")[-1].strip() for output in outputs_high_temp
    ] + [
        outputs_low_temp[0].split("Answer:")[-1].strip()
    ]

    return responses

# main function to to load questions and gerate answers and save them to a file

In [8]:
# Main function
def main():

    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:
      # Load the XQuAD dataset for Arabic (ar)
      xquad_dataset = load_dataset("xquad", "xquad.ar")

      # Load the validation set (the only available split)
      test_set = xquad_dataset['validation']

      print("File found")
      # Create an empty list to store the data
      data = []

      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['answers']


          # Append the data as a dictionary to the list
          data.append({
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })

    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=100).iterrows() :
        question = row['Question']
        original_answer = row.get('Answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)
#change the file name !!!!!!!!!!!!!!!!!!!
    filename = "generated_answers_jais6b7b_xquad.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    display(FileLink(filename))
    files.download(filename)

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/263k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

File found
    question_id                                           question  \
0           152  أي الدليل بين وضمن أصناف التعقيد من شأنه أن يد...   
1           588               متى افتُتح أول سوق داخلي في نيوكاسل؟   
2           338     من أين يرغب العلماء قياس إشعاع الغطاء النباتي؟   
3           999                            ماذا يعني كلوروبلاستيد؟   
4           206                كم توجد من منطقة حضرية ضخمة متوسعة؟   
..          ...                                                ...   
95          881        ماذا كان اسم أول جزء من سلسلة "الدكتور هو"؟   
96          990  أهداف من لا تزال غالباً تعارض الفريق الحكومي ا...   
97          962                    أين يعمل معظم الأطفال العاملين؟   
98          876            كم مرة يقوم "الدكتور هو" بالسفر وحيدًا؟   
99          218  ماسم فريق الرابطة الوطنية لكرة اليد بخلاف أناه...   

                                      original_answer  \
0   {'text': ['إن إثبات أن أي من هذه الأصناف غير م...   
1           {'text': ['1835'], 'an

/content/generated_answers_jais6b7b_xquad.json

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>